In [10]:
import click
import glob
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

In [14]:
#PASADO:      TempP0, TempP1, ..., TempP70, TempP71            (nx72)
#FUTURO:      TempF71, TempF72,..., TempF95                    (nx24)
#ALTITUD:     Altitud_Estacion - Altitud_Estacion_Prediccion   (nx1)
#MES:         Onehot encoding mes del año                      (nx12)
#REAL:        TempP71, TempP72,..., TempP95                    (nx24)



def crear_dataset(df_pasado, df_forecast, altitud_estacion, altitud_forecast):
    matriz_pasado = crear_pasado(df_pasado)
    assert len(df_pasado) == len(df_forecast),'La longitud de los dataset de train y forecast no coinciden{} - {}'.format(len(df_pasado), len(df_forecast))
    matriz_futuro = crear_futuro(df_forecast)
    matriz_altitud = np.repeat(altitud_estacion - altitud_forecast, len(df_train) - 96).reshape(-1,1)
    lb = LabelBinarizer()
    lb.fit([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
    matriz_mes = lb.transform(np.array(df_forecast['fecha'].dt.month.values)[96:])
    matriz_real = crear_real(df_train)
    #concatenar matrices
    return np.concatenate((matriz_pasado,matriz_futuro,matriz_altitud, matriz_mes, matriz_real), axis=1)
    
def crear_pasado(df_pasado):
    matriz_pasado = np.empty((len(df_pasado) - 72 - 24, 72)) # matriz vacia de len(df) x 72 
    for i in range(0, len(df_pasado) - 72 - 24):
        for j in range(0, 72):
            matriz_pasado[i, j] = df_pasado.iloc[i + j, 2] # coger columna 2 que es temperatura
    return matriz_pasado

def crear_futuro(df_forecast):
    matriz_futuro = np.empty((len(df_forecast) - 72 - 24, 24)) # matriz vacia de len(df) x 24
    for i in range(72, len(df_forecast) - 24):
        for j in range(0, 24):
            matriz_futuro[i-72, j] = df_forecast.iloc[i + j, 2] # coger columna 2 que es temperatura
    return matriz_futuro

def crear_real(df_train):
    matriz_real = np.empty((len(df_train) - 72 - 24, 24)) # matriz vacia de len(df) x 24
    for i in range(72, len(df_train) - 24):
        for j in range(0, 24):
            matriz_real[i-72, j] = df_train.iloc[i + j, 2] # coger columna 2 que es temperatura
    return matriz_real


lista_train = glob.glob("./train/ch*.csv")
print("lista_train: {}".format(lista_train))
lista_forecast = glob.glob("./forecast/ch*.csv")
print("lista_forecast: {}".format(lista_forecast))

headers = ['fecha', 'id', 'temp']
dtypes = {'fecha': 'str', 'id': 'float', 'temp': 'float'}
parse_dates = ['fecha']

df_forecast = pd.DataFrame()
for f in lista_forecast:
    df_forecast = pd.concat([df_forecast, pd.read_csv(f, decimal=".", sep=",", header=None, names=headers, dtype=dtypes, parse_dates=parse_dates)])
    altitud_forecast = int(f.split('A')[1])
    print("Altitud estacion forecast: {}".format(altitud_forecast))

train = np.empty((0,133))
for f in lista_train:
    df_train = pd.DataFrame() # Vaciar dataframe
    df_train = pd.read_csv(f, decimal=".", sep=",", header=None, names=headers, dtype=dtypes, parse_dates=parse_dates)
    altitud_estacion = int(f.split('A')[1])
    print("Altitud estacion: {}".format(altitud_estacion))
    datos_estacion = crear_dataset(df_train, df_forecast, altitud_estacion, altitud_forecast)
    #adjuntar matrices
    
    print("Estacion: {}, tamaño dataset antes: {}".format(f, train.shape))
    train = np.append(train, datos_estacion, axis=0)
    print("Estacion: {}, tamaño dataset despues: {}".format(f, train.shape))

#Normalizar columna de temperatura




lista_train: ['./train\\ch2465_A462A.csv', './train\\ch2809_A522A.csv', './train\\ch2810_A540A.csv', './train\\ch63_A505A.csv', './train\\ch88_A461A.csv']
lista_forecast: ['./forecast\\ch76_A353A.csv']
Altitud estacion forecast: 353
Altitud estacion: 462
Estacion: ./train\ch2465_A462A.csv, tamaño dataset antes: (0, 133)
Estacion: ./train\ch2465_A462A.csv, tamaño dataset despues: (8664, 133)
Altitud estacion: 522
Estacion: ./train\ch2809_A522A.csv, tamaño dataset antes: (8664, 133)
Estacion: ./train\ch2809_A522A.csv, tamaño dataset despues: (17328, 133)
Altitud estacion: 540
Estacion: ./train\ch2810_A540A.csv, tamaño dataset antes: (17328, 133)
Estacion: ./train\ch2810_A540A.csv, tamaño dataset despues: (25992, 133)
Altitud estacion: 505
Estacion: ./train\ch63_A505A.csv, tamaño dataset antes: (25992, 133)
Estacion: ./train\ch63_A505A.csv, tamaño dataset despues: (34656, 133)
Altitud estacion: 461
Estacion: ./train\ch88_A461A.csv, tamaño dataset antes: (34656, 133)
Estacion: ./train\ch88

In [17]:
temp_scaler = pickle.load(open('./mmscaler.pickle', 'rb'))
print(temp_scaler)
train[:, :96] = temp_scaler.transform(train[:, :96])
train[:, 108:132] = temp_scaler.transform(train[:, 108:132])


MinMaxScaler(copy=True, feature_range=(0, 1))


In [28]:
train=train2
alt_scaler = MinMaxScaler()
train[:, 96] = alt_scaler.fit_transform(train[:, 96].reshape(-1,1)).flatten()


(43320, 1)


In [27]:
print(train2[33000,96])

0.5569620253164558


In [8]:
print(train[:,:72])

[[0.50408759 0.49436381 0.49004214 ... 0.57539525 0.54784456 0.53055786]
 [0.49436381 0.49004214 0.48085857 ... 0.54784456 0.53055786 0.51867325]
 [0.49004214 0.48085857 0.45168725 ... 0.53055786 0.51867325 0.51002989]
 ...
 [0.55396694 0.53689632 0.50650052 ... 0.52119422 0.51363129 0.4796341 ]
 [0.53689632 0.50650052 0.4788778  ... 0.51363129 0.4796341  0.46760543]
 [0.50650052 0.4788778  0.4623834  ... 0.4796341  0.46760543 0.46245543]]


In [ ]:

def crear_muestras(df):
    """
    devuelve una matriz de la forma:
    Temp0, Temp1,..., Temp19, Temp20
    Temp1, Temp2,..., Temp20, Temp21
    
    Input: pandas dataframe
    Output: numpy
    """
    dato = np.empty((len(df) - 19, 20))
    for i in range(0, len(df) - 19):
        for j in range(0, 20):
            dato[i, j] = df.iloc[i + j, 1]
    
    return dato

# variables categoricas: utilizar la estacion del año
df['estacion'] = df['fecha'].dt.month.apply(lambda x: 'primavera' if x >= 3 and x < 6 
                                                 else 'verano' if x >=6 and x < 9
                                                 else 'otoño' if x >= 9 and x < 12
                                                 else 'invierno')

# montar el dataset
df = pd.concat( [df.loc[0:len(df)-20], pd.DataFrame(crear_muestras(df))], axis=1)
print("=========CABECERA===========\n", df.head())
print("===========COLA=============\n", df.tail())

df = df.drop(['fecha', 'temp'], axis=1)